In [66]:
import json
import os
import re
import pandas as pd

In [120]:
league = 21114

In [121]:
with open('data/bootstrap-static.json','r') as f:
    data = json.loads(f.read())

In [122]:
players = {}
for element in data['elements']:
    id = element['id']
    name = element['web_name']
    position = element['element_type']
    players[id] = {
        'name': name,
        'position': 'GK' if position == 1 else 'DEF' if position == 2 else 'MID' if position == 3 else 'FWD' 
    }

In [123]:
def list_files(directory):
    files = []
    for root, _, filenames in os.walk(directory):
        for filename in filenames:
            files.append(os.path.join(root, filename))
    return files

In [124]:
player_results = {}

file_list = list_files(f'./data/{league}/entries/')
for file_path in file_list:
    pattern = r"entry_(\d+)_(\d+)_picks"
    match = re.match(pattern, os.path.basename(file_path))

    id = match.group(1)
    week = match.group(2)

    with open(file_path,'r') as f:
        entry_data = json.loads(f.read())

    with open(f'./data/{league}/points/{week}_points.json', 'r') as f:
        points_data = json.loads(f.read())

    picks = entry_data['picks'][:11]
    for pick in picks:        
        element = pick['element']
        points = points_data['elements'][f'{element}']['stats']['total_points']
        player=players[element]
        
        if not element in player_results:
            player_results[element] = {
                'name': player['name'],
                'position': player['position'],
                'points': points,
            }
        else:
            current_points = player_results[element]['points']
            player_results[element]['points'] = current_points+points 

In [126]:
pd.DataFrame.from_dict(player_results, orient='index')\
    .sort_values(by='points', ascending=False)\
    .groupby('position').head(5).sort_values(['position', 'points'])

,name,position,points
16,Gabriel,DEF,124
377,Botman,DEF,126
10,White,DEF,134
285,Alexander-Arnold,DEF,150
357,Trippier,DEF,198
356,Wilson,FWD,137
40,Watkins,FWD,167
80,Toney,FWD,168
427,Kane,FWD,263
318,Haaland,FWD,272
